In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,3421
2,Huelva,Confirmados PDIA 14 días,1400
3,Huelva,Tasa PDIA 14 días,"274,1104626005643"
4,Huelva,Confirmados PDIA 7 días,874
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,64
629,Municipio de Huelva sin especificar,Total Confirmados,95
630,Municipio de Huelva sin especificar,Curados,9


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  3421.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  903.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 589 personas en los últimos 7 días 

Un positivo PCR cada 390 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,3421.0,874.0,1400.0,510743.0,171.123246,274.110463,468.0
Huelva-Costa,1717.0,416.0,674.0,288115.0,144.386790,233.934366,204.0
Huelva (capital),903.0,244.0,368.0,143663.0,169.841922,256.155029,131.0
Condado-Campiña,1055.0,229.0,365.0,155057.0,147.687625,235.397306,126.0
Sierra de Huelva-Andévalo Central,554.0,165.0,297.0,67571.0,244.187595,439.537671,75.0
Almonte,157.0,59.0,91.0,24191.0,243.892357,376.172957,32.0
Valverde del Camino,91.0,50.0,59.0,12820.0,390.015601,460.218409,27.0
Lepe,256.0,61.0,111.0,27431.0,222.376144,404.651671,26.0
Gibraleón,85.0,30.0,46.0,12607.0,237.963036,364.876656,19.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),32.0,4.0,20.0,516.0,775.193798,3875.968992,1.0
Cortegana,109.0,40.0,96.0,4666.0,857.265324,2057.436777,9.0
Escacena del Campo,41.0,21.0,23.0,2284.0,919.439580,1007.005254,16.0
Almonaster la Real,18.0,7.0,16.0,1819.0,384.826828,879.604178,1.0
Cerro de Andévalo (El),25.0,3.0,20.0,2364.0,126.903553,846.023689,1.0
Villarrasa,45.0,13.0,18.0,2176.0,597.426471,827.205882,9.0
Linares de la Sierra,2.0,1.0,2.0,263.0,380.228137,760.456274,0.0
Calañas,22.0,18.0,19.0,2791.0,644.930133,680.759584,12.0
Palma del Condado (La),159.0,37.0,61.0,10761.0,343.834216,566.861816,17.0
